# MesoNet 

MesoNet can be used either through a GUI or the command line interface (CLI). This notebook will make use of the CLI. 

While MesoNet can be used through 5 different "approaches", this notebook will focus on the following two:

ATLAS -> BRAIN: 
- Given a pre-trained DeepLabCut model that was trained to associate anatomical landmarks with corresponding points on atlases of brain regions, this approach registers an atlas of brain regions to the fixed brain imaging data using affine transformations. This approach is useful if your data has common anatomical landmarks and is the most robust to variations in image quality and orientation within your data.

BRAIN -> ATLAS:
- Given a pre-trained DeepLabCut model that was trained to associate anatomical landmarks with corresponding points on atlases of brain regions, the brain imaging data is fixed onto an atlas of brain regions using affine transformations. This approach is useful if you would like to normalize your brain images to a common template based on anatomical landmarks.




In [28]:
# make sure to activate your DEEPLABCUT environment before processing any data. 
!activate DEEPLABCUT


(DEEPLABCUT) c:\Users\bbettl\PycharmProjects\wfield_pipeline>conda.bat activate DEEPLABCUT 


In [29]:
# set verbose to True if you want to get feedback from each cell
verbose = True

In [30]:
# import all necessary modules

import mesonet
from data_loading_functions import *
import numpy as np
from datetime import datetime
import os 
import cv2
import shutil
import matplotlib as mpl 
from PIL import Image
from tqdm import tqdm
import glob

---
# Data Preparation: 
### dat or .npy --> .png

MesoNet requires that images be in an 8-bit (value 0-255) .png files in order to be used. 
The pipeline however currently allows a series of possible options. 

All input data for a single session / processing run must be in a single directory containing one of the following: 
1. a single .dat file
2. a series of .png brain images (may also contain a .dat file but NO .npy files)

An output directory within `Data_outputs` will be created with the current date and time in order to keep track of all processing. 

In [8]:
# add the path to the input and output directories as specified above
input_dir = "Data_inputs"
output_dir = "Data_outputs"

In [24]:
# helper function to move .npy files from the input_dir directory into a np_files directory
# helper function to remove all .png files from input_dir directory. Used for debugging and testing

def move_np_files(source_dir, target_dir):
    np_file_num = len(glob.glob((os.path.join(source_dir, '*.npy'))))

    # if there are no npy files, return funtion
    if np_file_num != 0:
        # check if a np_files directory exists. If not, make one
        if not os.path.exists(target_dir): os.makedirs(target_dir) 
        
        # iterate through all .npy files and move them to the target_dir
        [shutil.move(file, os.path.join(target_dir, os.path.basename(file))) for file in glob.glob(os.path.join(source_dir, '*.npy'))]
        
        return np_file_num
    else:
        return 0
    
def remove_png_files(source_dir):
    files_to_remove = glob.glob(os.path.join(source_dir, '*.png'))
    file_num = len(files_to_remove)

    # Iterate through all .png files and remove them with a progress bar
    for im in tqdm(files_to_remove, desc="Removing .png images", unit="image"):
        os.remove(im)
    
    if verbose:
        print(f"Finished removing all {file_num} .png files from {source_dir}")

    return None

In [27]:
# our pipeline works with a stack of np arrays representing the wfield images.
# prepare the data as it would appear after coming out of the preprocessing pipeline.
# For this example, we will load only 500 of the original 10000+ brain images. 

# check if there are .png images already present in the Data_inputs folder.
if len(glob.glob(f"{input_dir}/*.png")) != 0: # there are images present
    if verbose: print("Make sure images are 8-bit (0-255 value) before proceeding.")

else: # There must be a .dat file
    # make a numpy array from the .dat file
    np_img_stack = load_dat_frames(filename=glob.glob(os.path.join(input_dir, '*.dat'))[0])
    if verbose: print(f"numpy stack successfully loaded from .dat file. It has shape: {np_img_stack.shape}.")

    # make sure the array is of correct dimension: we only want access to the blue frames (single channel)
    if len(np_img_stack.shape) == 4:
        if np_img_stack.shape[1] != 1:
            np_img_stack = np_img_stack[:,0,...]
        np_img_stack = np.squeeze(np_img_stack, axis=1)
    
    # TODO: remove the lightweight option when making full version of this code. 
    np_img_stack = np_img_stack[:500,...]

    # make sure all array is of type uint8 with values 0-255
    if np_img_stack.dtype == 'uint16':
        # normalize the values to be between 0-1
        normalized_stack = (np_img_stack - np.min(np_img_stack)) / (np.max(np_img_stack) - np.min(np_img_stack))
        # scale to range 0-255
        scaled_stack = normalized_stack * 255.0
        # convert to uint8
        np_img_stack = scaled_stack.astype(np.uint8)
    
    # Convert the NumPy stack into a series of .png images with a progress bar
    for i, im in enumerate(tqdm(np_img_stack, desc="Saving images as .png", unit="image")):
        im_file_name = os.path.join(input_dir, f"frame_{i}.png")
        Image.fromarray(im).save(im_file_name)
    
    if verbose: print(f"{i+1} .png files have been created in the {input_dir} direcotry.")


# make sure there are no .npy files in the Data_inputs directory as this will impact the MesoNet processing
np_file_num = move_np_files(input_dir, "np_files")
if verbose:
    if np_file_num > 0: print(f"{np_file_num} numpy files were moved from {input_dir} to the np_files directory.")



numpy stack successfully loaded from .dat file. It has shape: (11220, 1, 640, 540).


Saving images: 100%|██████████| 500/500 [00:19<00:00, 26.25image/s]

500 .png files have been created in the Data_inputs direcotry.


In [26]:
# run this cell to clear all the .png images in the Data_inputs folder
remove_png_files(input_dir)

Removing .png images: 100%|██████████| 500/500 [00:00<00:00, 8432.46image/s]

Finished removing all 500 .png files from Data_inputs


---
# MesoNet Models

- U-Net (.hdf5)
- VoxelMorph (.h5) 

All models need to be in a folder called `models` within the *mesonet* directory of the MesoNet git repo.

### Configuration files:

The configuration file is used to customize the MesoNet analysis. It is created in the `output_dir` location 
The full list of configuration parameters can be found [here](https://github.com/bf777/MesoNet/wiki/Config-File-Guide).


##### Model Training params of interest: 

- <ins>**epochs**</ins> default 60: The number of epochs for which the model will be trained. You may wish to reduce this number if you're updating an existing model.

- <ins>**log_folder**</ins> folder to which logging data should be output (and where this config file will be saved)

- <ins>**steps_per_epoch**</ins> default 300: The number of steps per training epoch. Influences the pace at which the model is trained.



#### Model Testing params of interest: 

- <ins>**model**</ins> location (within the MesoNet repository) of a U-Net model to be used for finding the boundaries of the brain region (as the default model does), or (if you have a specially trained model for this purpose) segmenting the entire brain into regions without the need for atlas alignment. Only choose another model if you have another model that you would like to use for segmenting the brain.

- <ins>**threshold**</ins> adjusts the sensitivity of the algorithm used to define individual brain regions from the brain atlas. Changing this number may significantly change the quality of the brain region predictions
    - increasing this number causes each brain region contour to be smaller (less like the brain atlas)
    - decreasing this number causes each brain region contour to be larger (more like the brain atlas).

- <ins>**olfactory_check**</ins> when True, draws olfactory bulb contours on brain image

- <ins>**align_once**</ins> if True, carries out all alignments based on the alignment of the first atlas and brain. This can save time if you have many frames of the same brain with a fixed camera position.

- <ins>**atlas_label_list**</ins> list of aligned atlases in which each brain region is filled with a unique numeric label. Allows for consistent identification of brain regions across images. If original_label is True, this is an empty list.

- <ins>**landmark_arr**</ins> The default number and order of landmarks to be used for the full alignment of a standard brain atlas to a brain image. Change what is contained in this list to change the landmarks used. When the default DeepLabCut model with nine landmarks, the landmarks are
    - 0: Anterolateral tip of the left parietal bone.
    - 1: Left frontal pole.
    - 2: Posterior tip of the left retrosplenial region.
    - 3: Cross point between the median line and the line which connects the left and right frontal pole.
    - 4: Bregma (centre point of cortex)
    - 5: Anterior tip of the interparietal bone.
    - 6: Anterolateral tip of the right parietal bone.
    - 7: Right frontal pole.
    - 8: Posterior tip of the right retrosplenial region.

---
## Model Training:

We will be using the same `input_dir` and `output_dir` variables as previously loaded. Make sure these are correctly intiialized before proceeding. 

---
## Model Testing:

We will be using the same `input_dir` and `output_dir` variables as previously loaded. Make sure these are correctly intiialized before proceeding. 

In [8]:
# prepare a configuration file that will be used to ___
config_file = mesonet.config_project(input_file, output_file, 'test')

if verbose: "config_file has been created with input_file and output_file paths"

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x107e4e910>>
Traceback (most recent call last):
  File "/Users/brune/anaconda3/envs/DEEPLABCUT/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
mesonet.predict_regions(config_file)
mesonet.predict_dlc(config_file)